In [ ]:
pip install transformers

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import torch
import torch.nn as nn

import transformers
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip drive/MyDrive/MSR_data_0831/data_0831.zip

Archive:  drive/MyDrive/MSR_data_0831/data_0831.zip
replace __MACOSX/._0831? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: __MACOSX/._0831         
  inflating: 0831/.DS_Store          
  inflating: __MACOSX/0831/._.DS_Store  
  inflating: 0831/I_tensor20210829   
  inflating: __MACOSX/0831/._I_tensor20210829  
  inflating: 0831/df_csv_joined20180831.csv  
  inflating: __MACOSX/0831/._df_csv_joined20180831.csv  
  inflating: 0831/data_tensor_fnl20210831  
  inflating: __MACOSX/0831/._data_tensor_fnl20210831  


In [ ]:
!unzip drive/MyDrive/MSR_data_0831/data_0831.zip > /dev/null

A
replace __MACOSX/._0831? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
#url_df_read = pd.read_csv('drive/MyDrive/MSR_data_0831/df_csv_joined20180828.csv', index_col=[0])
#url_df_read

In [ ]:
#MSR_df_read = pd.read_csv('drive/MyDrive/MSR_data_0831/MSR_text_2.csv', index_col=[0])
#MSR_df_read

In [ ]:
df_csv_joined = pd.read_csv('drive/MyDrive/MSR_data_0831/df_csv_joined20180831.csv')
df_csv_joined

,Unnamed: 0,level_0,Unnamed: 0.1,index,Answer ID,Answer Page,Metric,Company,Value,label,Text,Source Page,URL_link
0,0,0,0,0,6382834,https://wikirate.org/~6382834,Walk Free Foundation+MSA whistleblowing mechan...,S A Brain Company Ltd,"Whistleblower protection (direct employees), W...",1,10/19/2020\n\nSlavery and Human trafﬁcking | S...,https://wikirate.org/~6375302,https://dq06ugkuram52.cloudfront.net/files/637...
1,1,1,1,1,6368402,https://wikirate.org/~6368402,Walk Free Foundation+MSA whistleblowing mechan...,Pension Protection Fund,"Whistleblower protection (direct employees), I...",1,\n\n \n\n \n\n \n\n \n\n \n\n \n\nModern Slav...,https://wikirate.org/~6355100,https://dq06ugkuram52.cloudfront.net/files/635...
2,2,2,2,2,6380781,https://wikirate.org/~6380781,Walk Free Foundation+MSA whistleblowing mechan...,Hall & Woodhouse Limited,Whistleblower protection (direct employees),1,10/19/2020\n\nSlavery | Hall and Woodhouse\n\n...,https://wikirate.org/~6375083,https://dq06ugkuram52.cloudfront.net/files/637...
3,3,3,3,3,6364508,https://wikirate.org/~6364508,Walk Free Foundation+MSA whistleblowing mechan...,Greene King,Whistleblower protection (direct employees),1,Greene King | MODERN SLAVERY STATEMENT\n\nhttp...,https://wikirate.org/~6357513,https://dq06ugkuram52.cloudfront.net/files/635...
4,4,4,4,4,6379525,https://wikirate.org/~6379525,Walk Free Foundation+MSA whistleblowing mechan...,Charles Wells,Whistleblower protection (direct employees),1,Charles Wells Ltd \n \nModern Slavery and Huma...,https://wikirate.org/~6354153,https://dq06ugkuram52.cloudfront.net/files/635...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1637,1650,1651,1651,1713,7311793,https://wikirate.org/~7311793,Walk Free Foundation+MSA whistleblowing mechan...,ECO Animal Health Group Plc,No,0,\n\n \nModern Slavery Act 2015 Statement \n...,https://wikirate.org/~7307585,https://dq06ugkuram52.cloudfront.net/files/730...
1638,1651,1652,1652,1714,7310878,https://wikirate.org/~7310878,Walk Free Foundation+MSA whistleblowing mechan...,Victoria Plum Limited,No,0,Upto 60% off + 20% off ends\n\n \n\n \n\nSearc...,https://wikirate.org/~7307593,https://dq06ugkuram52.cloudfront.net/files/730...
1639,1652,1653,1653,1715,7310802,https://wikirate.org/~7310802,Walk Free Foundation+MSA whistleblowing mechan...,Creagh Concrete Products Limited,Whistleblower protection (direct employees),1,\n\nSlavery and Human Trafficking Transparenc...,https://wikirate.org/~7307608,https://dq06ugkuram52.cloudfront.net/files/730...
1640,1653,1654,1654,1716,7311831,https://wikirate.org/~7311831,Walk Free Foundation+MSA whistleblowing mechan...,Reward Gateway (UK) Ltd,"Hotline (direct employees), Whistleblower prot...",1,Reward Gateway Slavery\nand Human Trafficking\...,https://wikirate.org/~7307616,https://dq06ugkuram52.cloudfront.net/files/730...


In [ ]:
short_doc = []
for i in range(len(df_csv_joined['Text'])):
  if len(df_csv_joined['Text'][i]) < 1500:
    short_doc.append(i)


In [ ]:
df = df_csv_joined.drop(short_doc)
df

,Unnamed: 0,level_0,Unnamed: 0.1,index,Answer ID,Answer Page,Metric,Company,Value,label,Text,Source Page,URL_link
0,0,0,0,0,6382834,https://wikirate.org/~6382834,Walk Free Foundation+MSA whistleblowing mechan...,S A Brain Company Ltd,"Whistleblower protection (direct employees), W...",1,10/19/2020\n\nSlavery and Human trafﬁcking | S...,https://wikirate.org/~6375302,https://dq06ugkuram52.cloudfront.net/files/637...
1,1,1,1,1,6368402,https://wikirate.org/~6368402,Walk Free Foundation+MSA whistleblowing mechan...,Pension Protection Fund,"Whistleblower protection (direct employees), I...",1,\n\n \n\n \n\n \n\n \n\n \n\n \n\nModern Slav...,https://wikirate.org/~6355100,https://dq06ugkuram52.cloudfront.net/files/635...
2,2,2,2,2,6380781,https://wikirate.org/~6380781,Walk Free Foundation+MSA whistleblowing mechan...,Hall & Woodhouse Limited,Whistleblower protection (direct employees),1,10/19/2020\n\nSlavery | Hall and Woodhouse\n\n...,https://wikirate.org/~6375083,https://dq06ugkuram52.cloudfront.net/files/637...
3,3,3,3,3,6364508,https://wikirate.org/~6364508,Walk Free Foundation+MSA whistleblowing mechan...,Greene King,Whistleblower protection (direct employees),1,Greene King | MODERN SLAVERY STATEMENT\n\nhttp...,https://wikirate.org/~6357513,https://dq06ugkuram52.cloudfront.net/files/635...
4,4,4,4,4,6379525,https://wikirate.org/~6379525,Walk Free Foundation+MSA whistleblowing mechan...,Charles Wells,Whistleblower protection (direct employees),1,Charles Wells Ltd \n \nModern Slavery and Huma...,https://wikirate.org/~6354153,https://dq06ugkuram52.cloudfront.net/files/635...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1637,1650,1651,1651,1713,7311793,https://wikirate.org/~7311793,Walk Free Foundation+MSA whistleblowing mechan...,ECO Animal Health Group Plc,No,0,\n\n \nModern Slavery Act 2015 Statement \n...,https://wikirate.org/~7307585,https://dq06ugkuram52.cloudfront.net/files/730...
1638,1651,1652,1652,1714,7310878,https://wikirate.org/~7310878,Walk Free Foundation+MSA whistleblowing mechan...,Victoria Plum Limited,No,0,Upto 60% off + 20% off ends\n\n \n\n \n\nSearc...,https://wikirate.org/~7307593,https://dq06ugkuram52.cloudfront.net/files/730...
1639,1652,1653,1653,1715,7310802,https://wikirate.org/~7310802,Walk Free Foundation+MSA whistleblowing mechan...,Creagh Concrete Products Limited,Whistleblower protection (direct employees),1,\n\nSlavery and Human Trafficking Transparenc...,https://wikirate.org/~7307608,https://dq06ugkuram52.cloudfront.net/files/730...
1640,1653,1654,1654,1716,7311831,https://wikirate.org/~7311831,Walk Free Foundation+MSA whistleblowing mechan...,Reward Gateway (UK) Ltd,"Hotline (direct employees), Whistleblower prot...",1,Reward Gateway Slavery\nand Human Trafficking\...,https://wikirate.org/~7307616,https://dq06ugkuram52.cloudfront.net/files/730...


In [ ]:
data_tensor = torch.load('drive/MyDrive/MSR_data_0831/data_tensor_fnl20210831')
data_tensor_np = data_tensor.detach().numpy()
data_tensor_np1 = np.delete(data_tensor_np, short_doc, axis=0)

print(data_tensor_np1.shape)

data_tensor_np2 = torch.tensor(data_tensor_np1)

print(data_tensor_np2.size())

(1624, 200, 768)
torch.Size([1624, 200, 768])


#### I Tensor

In [ ]:
I_tensor = torch.load('drive/MyDrive/MSR_data_0831/I_tensor20210829')
I_tensor_np = I_tensor.detach().numpy()
I_tensor_np1 = np.delete(I_tensor_np, short_doc, axis=0)

print(I_tensor_np1.shape)

I_tensor1 = torch.tensor(I_tensor_np1)

print(I_tensor1.size())


(1624, 1, 200)
torch.Size([1624, 1, 200])


### Model training

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
label = df['label'].tolist()
len(label)

1624

In [ ]:
label = df['label'].tolist()

train_x = data_tensor_np2[:1000]
train_x = train_x.permute(0,2,1)
train_y = torch.tensor(label[:1000])
train_i = I_tensor1[:1000]

val_x = data_tensor_np2[1000:]
val_x = val_x.permute(0,2,1)
val_y = torch.tensor(label[1000:])
val_i = I_tensor1[1000:]

test_x = data_tensor_np2[1000:]
test_x = test_x.permute(0,2,1)
test_y = torch.tensor(label[1000:])
test_i = I_tensor1[1000:]

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 50

train_data = TensorDataset(train_x, train_y, train_i)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler = train_sampler, batch_size=batch_size, shuffle=False)

val_data = TensorDataset(val_x, val_y, val_i)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size, shuffle=False)

In [ ]:
class model(nn.Module):

    def __init__(self):
        
        super(model, self).__init__()
                        
        self.conv1 = nn.Conv1d(in_channels = 768, out_channels=1, kernel_size=1, stride=1, padding=0)
        
        self.conv2 = nn.Conv1d(in_channels = 768, out_channels=512, kernel_size=1, stride=1, padding=0)
        
        self.fc1 = nn.Linear(512, 512)
        
        self.fc2 = nn.Linear(512, 2)
                
        self.softmax1 = nn.Softmax(dim=-1)
        
        self.relu =  nn.ReLU()
        
        self.dropout = nn.Dropout(0.2)
        

    #define the forward pass
    def forward(self, x, y, i):
        
        lbda = 5
                        
        alpha = self.conv1(x) #(btch_sz, 768, 200)-> (btch_sz, 1, 200)
        
        x_2 = self.conv2(x)
        
        x_2 = self.relu(x_2)
                                                        
        alpha1 = alpha + lbda * i #(btch_sz, 1, 200) + (btch_sz,1, 200)*(btch_sz, 1, 200) 
                                
        alpha_prime = self.softmax1(alpha1) #(btch_sz, 1, 200)
                        
        alpha_prime_permute = alpha_prime.permute(0,2,1) #(btch_sz, 200, 1) 
                                        
        h = torch.matmul(x_2, alpha_prime_permute)  #(btch_sz, 768, 200)*(btch_sz, 200, 1) = (btch_sz, 768, 1)
                                
        h = h.view(h.shape[0], -1) #(btch_sz, 768)
                        
        output = self.relu(self.fc1(h)) #(btch_sz, 768, 512) 
        
        output = self.dropout(output)
        
        output = self.fc2(output)
                                 
        return output

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
 
class_wts = compute_class_weight('balanced', classes= torch.unique(train_y).tolist(), y=train_y.tolist())

print(class_wts)

weights = torch.tensor(class_wts, dtype = torch.float)
weights = weights.to(device)


[1.03519669 0.96711799]


In [ ]:
from torch import optim

Model = model()

Model = Model.to(device)

cross_entropy = nn.CrossEntropyLoss(weight=weights)

from transformers import AdamW

optimizer = optim.AdamW(Model.parameters(), lr = 0.0005, weight_decay=0.01)

epochs = 200


In [ ]:
def train():
    Model.train()
    
    total_loss = 0

    train_correct = 0
    train_total = 0
    
    for step, batch in enumerate(train_dataloader):

        batch = [r.to(device) for r in batch]
 
        x, y, i = batch

        Model.zero_grad()        

        preds = Model(x, y, i)

        loss = cross_entropy(preds, y)

        total_loss = total_loss + loss.item()
     
        loss.backward()

        #torch.nn.utils.clip_grad_norm_(Model.parameters(), 1.0) 

        optimizer.step()  # update parameters

        _, predicted = torch.max(preds.data, 1)
        train_total += y.size(0)
        train_correct += (predicted == y).sum().item()

        preds=preds.detach().cpu().numpy()   # if model predictions are stored on GPU, push it to CPU
        
    avg_loss = total_loss / len(train_dataloader) # compute the training loss of the epoch
    train_acc = train_correct/train_total
  
    return avg_loss, train_acc #returns train loss and predictions


In [ ]:
def evaluate():
    
    print("\nEvaluating...")
    Model.eval()   # deactivate dropout layers
    
    total_loss = 0

    val_total = 0
    val_correct = 0

    for step,batch in enumerate(val_dataloader):   # iterate over batches
        
        batch = [t.to(device) for t in batch] # push the batch to gpu

        x, y, i = batch
        
        with torch.no_grad(): # deactivate autograd
            
            preds = Model(x, y, i)
            
            loss = cross_entropy(preds,y)

            total_loss = total_loss + loss.item()

            _, predicted = torch.max(preds.data, 1)
            val_total += y.size(0)
            val_correct += (predicted == y).sum().item()

            preds = preds.detach().cpu().numpy()
            
    avg_loss = total_loss / len(val_dataloader) 
    val_acc = val_correct/val_total

    return avg_loss, val_acc

In [ ]:
import numpy as np

#best_valid_loss = float('inf')  # set initial loss to infinite

best_valid_acc = 0

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

train_acces = []
val_acces = []

for epoch in range(epochs): #for each epoch
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

    train_loss, train_acc = train() #train model
    valid_loss, val_acc = evaluate() #evaluate model
    
    #if valid_loss < best_valid_loss: #save the best model
        #best_valid_loss = valid_loss
        #torch.save(Model.state_dict(), 'saved_weights.pt')

    if val_acc > best_valid_acc: 
        best_valid_acc = val_acc
        torch.save(Model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    
    train_acces.append(train_acc)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    val_acces.append(val_acces)
    
    print('Train acc', np.round(train_acc, 3), 'Val acc', np.round(val_acc, 3))
    print('Train loss', np.round(train_loss, 3), 'Val loss', np.round(valid_loss, 3))



 Epoch 1 / 200

Evaluating...
Train acc 0.647 Val acc 0.678
Train loss 0.614 Val loss 0.611

 Epoch 2 / 200

Evaluating...
Train acc 0.731 Val acc 0.691
Train loss 0.526 Val loss 0.638

 Epoch 3 / 200

Evaluating...
Train acc 0.75 Val acc 0.688
Train loss 0.509 Val loss 0.637

 Epoch 4 / 200

Evaluating...
Train acc 0.761 Val acc 0.696
Train loss 0.497 Val loss 0.63

 Epoch 5 / 200

Evaluating...
Train acc 0.769 Val acc 0.686
Train loss 0.486 Val loss 0.651

 Epoch 6 / 200

Evaluating...
Train acc 0.781 Val acc 0.689
Train loss 0.472 Val loss 0.681

 Epoch 7 / 200

Evaluating...
Train acc 0.779 Val acc 0.7
Train loss 0.478 Val loss 0.662

 Epoch 8 / 200

Evaluating...
Train acc 0.791 Val acc 0.7
Train loss 0.461 Val loss 0.704

 Epoch 9 / 200

Evaluating...
Train acc 0.779 Val acc 0.692
Train loss 0.471 Val loss 0.729

 Epoch 10 / 200

Evaluating...
Train acc 0.774 Val acc 0.679
Train loss 0.471 Val loss 0.717

 Epoch 11 / 200

Evaluating...
Train acc 0.785 Val acc 0.681
Train loss 0.

In [ ]:
path = 'saved_weights.pt'
Model.load_state_dict(torch.load(path))

print('Train_report:')
with torch.no_grad():
  train_preds = Model(train_x.to(device), train_y.to(device), train_i.to(device))
  train_preds = train_preds.detach().cpu().numpy()

train_preds = np.argmax(train_preds, axis = 1)
print(classification_report(train_y, train_preds))


print('Val_report:')
with torch.no_grad():
  val_preds = Model(val_x.to(device), val_y.to(device), val_i.to(device))
  val_preds = val_preds.detach().cpu().numpy()

val_preds = np.argmax(val_preds, axis = 1)
print(classification_report(val_y, val_preds))

print('Test_report:')
with torch.no_grad():
  test_preds = Model(test_x.to(device), test_y.to(device), test_i.to(device))
  test_preds = test_preds.detach().cpu().numpy()

test_preds = np.argmax(test_preds, axis = 1)
print(classification_report(test_y, test_preds))

Train_report:
              precision    recall  f1-score   support

           0       0.79      0.94      0.86       483
           1       0.93      0.76      0.84       517

    accuracy                           0.85      1000
   macro avg       0.86      0.85      0.85      1000
weighted avg       0.86      0.85      0.85      1000

Val_report:
              precision    recall  f1-score   support

           0       0.61      0.78      0.69       247
           1       0.82      0.68      0.74       377

    accuracy                           0.72       624
   macro avg       0.72      0.73      0.71       624
weighted avg       0.74      0.72      0.72       624

Test_report:
              precision    recall  f1-score   support

           0       0.61      0.78      0.69       247
           1       0.82      0.68      0.74       377

    accuracy                           0.72       624
   macro avg       0.72      0.73      0.71       624
weighted avg       0.74      0.72  

In [ ]:
x = list(range(0,200))
plt.figure(figsize=(10,6))
plt.plot(x, train_acces)
plt.plot(x, val_acces)
plt.plot(x, train_losses)
plt.plot(x, valid_losses)

plt.xlabel('epochs')
plt.ylabel('Accuracy')
plt.title('Model train and validation loss')